# Battle of the Neighborhoods

### Coursera Captone Project

## Introduction

As someone who will be moving to New Your City and is an avid coffee drinker it is very important to move to an area with a lot of coffe shops nearby.  In this project I will be comparing differenc areas of New York to determine which area has the best options avaialbe for a varity of coffee shops.

## Data

In order to answer the above questions, data on New York City neighborhoods, boroughs to include boundaries, latitude, longitude, restaurants, and restaurant ratings and tips are required.

New York City data containing the neighborhoods and boroughs, latitudes, and longitudes will be obtained from the data source: https://cocl.us/new_york_dataset

New York City data containing neighborhood boundaries will be obtained from the data source: https://data.cityofnewyork.us/City-Government/Borough-Boundaries/tqmj-j8zm

All data related to locations and quality of  coffee shops will be obtained via the FourSquare API utilized via the Request library in Python